In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

#### Используя файл Lesson_3_extended.ipynb (он в web3.zip в материалах):

1*. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.
3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).
4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

5*. Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.

6. Могла ли модель переобучиться? Почему?

In [2]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype=np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

In [3]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X, y)
model.predict(X)

array([ 0.40632829,  0.27334007,  1.05328664, -0.15160214,  0.46496879,
        0.32970504,  1.12205628,  0.17529032,  0.43957535,  0.88705137])

In [4]:
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

In [5]:
X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])

In [6]:
X_st

array([[ 1.        ,  1.        , -0.97958969,  1.        ],
       [ 1.        ,  1.        , -0.56713087,  1.        ],
       [ 1.        ,  2.        , -0.46401617,  2.        ],
       [ 1.        ,  5.        , -0.77336028,  1.        ],
       [ 1.        ,  3.        ,  0.97958969,  2.        ],
       [ 1.        ,  0.        , -0.36090146,  1.        ],
       [ 1.        ,  5.        ,  1.08270439,  3.        ],
       [ 1.        , 10.        ,  2.11385144,  3.        ],
       [ 1.        ,  1.        , -1.08270439,  1.        ],
       [ 1.        ,  2.        ,  0.05155735,  2.        ]])

In [7]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [8]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [9]:
y_pred = sigmoid(model.predict(X))
y_pred

array([0.60020715, 0.5679127 , 0.74140552, 0.46217189, 0.61419225,
       0.58168761, 0.75436994, 0.54371071, 0.60815784, 0.7082813 ])

In [10]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

#### 1*. Модифицируем функцию calc_logloss:

In [11]:
'''Идея модификации функции в том, чтобы передавать не 0 или 1, а крайне близкое к ним значение,
например 0.000001 / 0.999999. Не считать ошибку на примерах, к примеру, с нулями, которые наиболее
важные для нас-это грубая ошибка''' 

def calc_logloss_mod(y, y_pred):
    y_pred_res=np.where(y_pred==1, y_pred-1e-09, np.where(y_pred==0, y_pred+1e-09, y_pred))
    err = - np.mean(y * np.log(y_pred_res) + (1.0 - y) * np.log(1.0 - y_pred_res))
    return err

In [12]:
# Чтобы проверить работоспособность функции, заменим рандомные значения в векторе предсказанных значений на нули и единицы

y_pred_demo = np.array([0.60020715, 0. , 0.74140552, 0.46217189, 1.,
       0.58168761, 0.75436994, 0.54371071, 0., 0.7082813])

calc_logloss_mod(y, y_pred_demo)

2.4842439777208694

In [13]:
# Немодифицированная (исходная) функция при всех равных возвращает ошибку (nan)

calc_logloss(y, y_pred_demo)

<ipython-input-7-7d5907c1794a>:2: RuntimeWarning: divide by zero encountered in log
  err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
<ipython-input-7-7d5907c1794a>:2: RuntimeWarning: invalid value encountered in multiply
  err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))


nan

#### 2.

In [14]:
def eval_model(X, y, verbose=False, eta=1e-4, tol=0.00001):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    min_err = float('inf')  # зададим бесконечно большое начальное значение ошибки
    n_iter = 0  # отслеживаем количество итераций
    view_ind = 10**(-np.log10(tol)-2) if -np.log10(tol)-2>=1 else 1 # параметр кратности вывода промежуточных результатов
    stop_check = True
    errors = []  # добавим для визуализации кривой обучения
    
    while stop_check:
        n_iter += 1
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss_mod(y, y_pred) # заменим на модифицированную нами ранее функцию
        errors.append(err)
        if min_err - err > tol:  # контролируем текущее значение ошибки
            min_err = err
        else:  # если снижение прекратилось, останавливаемся.
            print(
                f'Stop descent! iteration: {n_iter}, weights: {W}, logloss: {min_err}')
            stop_check = False
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if verbose:
            if n_iter % view_ind == 0:
                print(n_iter, W, err)
    return W, min_err, n_iter

Теперь подберем лучшие параметры eta для модели

In [15]:
def get_best_params(X,y,args):
    best_params=[]
    for arg in args:
        W,err,n_iter = eval_model(X, y, eta=arg)
        best_params.append((arg,err,n_iter))
    best_params.sort(key=lambda x:x[1])
    print(f'best - alpha: {best_params[0][0]},\nresults:\nerr: {best_params[0][1]},\nn_iter: {best_params[0][2]}')
    return best_params[0]

In [16]:
alphas=np.arange(1,10)/10
best_params=get_best_params(X_st,y,alphas)
best_params

Stop descent! iteration: 5507, weights: [-7.85409116 -1.16596006 -1.51053115  6.85817166], logloss: 0.2644646885841898
Stop descent! iteration: 5473, weights: [-11.26122492  -1.42607616  -2.57916356   9.54824479], logloss: 0.227225466097774
Stop descent! iteration: 5668, weights: [-14.17179784  -1.67221877  -3.45623879  11.89872411], logloss: 0.2026696681492856
Stop descent! iteration: 5684, weights: [-16.49351659  -1.87498924  -4.14297044  13.78351526], logloss: 0.18614618122887375
Stop descent! iteration: 5588, weights: [-18.35029116  -2.03943006  -4.68623881  15.29282887], logloss: 0.17451429964096338
Stop descent! iteration: 5449, weights: [-19.88389849  -2.17629563  -5.13166996  16.53977688], logloss: 0.16583588446774875
Stop descent! iteration: 4, weights: [ 0.20089413 -0.9754314   0.73619273  1.29296406], logloss: 0.5606348884004944
Stop descent! iteration: 4, weights: [ 0.16967474 -1.19422836  0.72412524  1.26942358], logloss: 0.6239910394972162
Stop descent! iteration: 4, weig

(0.6, 0.16583588446774875, 5449)

Лучший результат получаем при eta=0.6

In [17]:
W = eval_model(X_st, y, eta=0.6, tol=0.00001, verbose=True)

1000 [-8.22705654 -1.19179224 -1.63103389  7.14617725] 0.2597559875895107
2000 [-11.81693228  -1.47202752  -2.74842535   9.99506855] 0.2221605978051024
3000 [-14.60325068  -1.70958893  -3.58457685  12.24861365] 0.19942508537234374
4000 [-16.96660182  -1.91673038  -4.28184084  14.16798183] 0.1830620510486847
5000 [-19.03415014  -2.10036075  -4.88520289  15.84885732] 0.17054733057426585
Stop descent! iteration: 5449, weights: [-19.88389849  -2.17629563  -5.13166996  16.53977688], logloss: 0.16583588446774875


#### 3.

In [18]:
def calc_pred_proba(x, w): 
    pred_proba = sigmoid(np.dot(x, w))
    return pred_proba

In [19]:
W,_err,_it = eval_model(X_st, y, eta=0.6, verbose=True)

1000 [-8.22705654 -1.19179224 -1.63103389  7.14617725] 0.2597559875895107
2000 [-11.81693228  -1.47202752  -2.74842535   9.99506855] 0.2221605978051024
3000 [-14.60325068  -1.70958893  -3.58457685  12.24861365] 0.19942508537234374
4000 [-16.96660182  -1.91673038  -4.28184084  14.16798183] 0.1830620510486847
5000 [-19.03415014  -2.10036075  -4.88520289  15.84885732] 0.17054733057426585
Stop descent! iteration: 5449, weights: [-19.88389849  -2.17629563  -5.13166996  16.53977688], logloss: 0.16583588446774875


In [20]:
y_pred_proba = calc_pred_proba(X_st, W)
y_pred_proba

array([3.79075791e-01, 6.84786993e-02, 9.99986668e-01, 3.50840343e-05,
       8.37531436e-01, 1.83584725e-01, 9.99998324e-01, 5.33913207e-02,
       5.08932298e-01, 9.99812077e-01])

#### 4.

In [21]:
def calc_pred(x,w, prob_lim=0.5):    # установим порог вероятности, при превышении которого, объект будет относиться к классу 1
    pred_proba = sigmoid(np.dot(x, w))
    pred=np.zeros_like(pred_proba)
    for idx, prob in enumerate(pred_proba):
        if prob>prob_lim:
            pred[idx]=1
    return pred

In [22]:
y_pred=calc_pred(X_st, W)
y_pred

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

#### 5*.

In [23]:
# Для проверки верности самописных функций импортируем следующие метрики из библиотеки sklearn

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [24]:
def east_accuracy(real, pred):
    all_res=len(real)
    trues=0
    for i in range(all_res):
        trues += int(real[i]==pred[i])
    return trues/all_res

In [25]:
east_accuracy(y,y_pred)

1.0

In [26]:
# проверка

accuracy_score(y,y_pred)

1.0

In [27]:
def east_confusion_matrix(real, pred):
    n_classes=len(np.unique(real))
    all_res=len(real)
    conf_matr=np.zeros((n_classes,n_classes), dtype='int')
    for i in range(all_res):
        conf_matr[int(real[i])][int(pred[i])] += 1
    return conf_matr

In [28]:
east_confusion_matrix(y, y_pred)

array([[5, 0],
       [0, 5]])

In [29]:
# проверка

confusion_matrix(y, y_pred)

array([[5, 0],
       [0, 5]], dtype=int64)

In [30]:
def east_precision(real, pred):
    tn, fp, fn, tp = east_confusion_matrix(real, pred).ravel()
    return tp/(tp+fp)

In [31]:
east_precision(y, y_pred)

1.0

In [32]:
# проверка

precision_score(y, y_pred)

1.0

In [33]:
def east_recall(real, pred):
    tn, fp, fn, tp = east_confusion_matrix(real, pred).ravel()
    return tp/(tp+fn)

In [34]:
east_recall(y, y_pred)

1.0

In [35]:
recall_score(y, y_pred)

1.0

In [36]:
def east_f1(real, pred, beta=1):
    return (1+beta**2)*east_precision(real, pred)*east_recall(y,y_pred)/(beta**2*east_precision(real, pred)+east_recall(y,y_pred))

In [37]:
east_f1(y, y_pred)

1.0

In [38]:
# проверка

f1_score(y,y_pred)

1.0

#### 6.

Можно предположить, что модель могла переобучиться, потому как модель логистической регрессии склонна к переобучению, в следствии того что мы используем сигмоиду для того, чтобы преобразовать в вероятность предсказания линейной модели. А из-за того, что сигмоида не имеет максимума и минимума, а только асимптоты в 0 и 1, градиентный спуск не может достичь оптимального решения с помощью градиентных шагов доводя веса до всё более экстремальных значений, пытаясь достичь нулевых потерь. При большой размерности данных вероятность этого, скорее всего, еще больше увеличится. Чтобы этого избежать в качестве способа регуляризации можно использовать ранний останов работы модели, а так же L1 и L2-регуляризацию.